## Interaction with openstreetmap data

In [ ]:
# imports
import networkx as nx
import osmnx as ox
import requests
import matplotlib.cm as cm
import matplotlib.colors as colors
ox.config(use_cache=True, log_console=True)
ox.__version__

### get some data about a certain area/city

In [ ]:
G = ox.graph_from_place('Danubyu, Ayeyarwady Region, Myanmar', network_type='walk')
fig, ax = ox.plot_graph(G)

In [ ]:
G_proj = ox.project_graph(G)
nodes_proj = ox.graph_to_gdfs(G_proj, edges=False)
graph_area_m = nodes_proj.unary_union.convex_hull.area
# street network area in km2
graph_area_m/1_000_000

### Street centrality

In [ ]:
# edge closeness centrality: convert graph to line graph so edges become nodes and vice versa
edge_centrality = nx.closeness_centrality(nx.line_graph(G))

In [ ]:
# list of edge values for the orginal graph
ev = [edge_centrality[edge + (0,)] for edge in G.edges()]

# color scale converted to list of colors for graph edges
norm = colors.Normalize(vmin=min(ev)*0.8, vmax=max(ev))
cmap = cm.ScalarMappable(norm=norm, cmap=cm.inferno)
ec = [cmap.to_rgba(cl) for cl in ev]

# color the edges in the original graph with closeness centralities in the line graph
fig, ax = ox.plot_graph(G, bgcolor='k', axis_off=True, fig_height=9, fig_width=16, node_size=0, edge_color=ec, edge_linewidth=1.5, edge_alpha=1)

### Building footprint in Danubyu area

In [ ]:
gdf = ox.footprints.footprints_from_place(place='Danubyu, Ayeyarwady Region, Myanmar')
gdf_proj = ox.project_gdf(gdf)
fig, ax = ox.footprints.plot_footprints(gdf_proj, bgcolor='black', color='yellow', figsize=(9,9),
                           show=True, filename='danubyu_bldgs', dpi=90)


In [ ]:
ox.plot_graph_folium(G, edge_color="green", edge_width=2)

### lets check what area is covered by buildings and how much percentage

In [ ]:
# calculate the area in projected units (m2) of each building footprint, then display first five
areas = gdf_proj.area
areas.head()

In [ ]:
# total area (sqm) covered by building footprints
total_building_area = sum(areas)
total_building_area

In [ ]:
# getting the area of danubyu(if you wonder why we didnt use the third code cell its because that one 
# is just a stret network not the district)
place = ox.gdf_from_place('Danubyu, Ayeyarwady Region, Myanmar')
place_proj = ox.project_gdf(place)
danubyu_area = place_proj.area.iloc[0]
danubyu_area

In [ ]:
# percentage
(total_building_area/danubyu_area) * 100

it seems to not be so far from the 2014 census considering that OSM data isnt that much accurate in rural area 
https://themimu.info/sites/themimu.info/files/documents/TspProfiles_Census_Danuphyu_2014_ENG.pdf

### Routing

In [ ]:
# get the nearest network node to each point
orig_node = ox.get_nearest_node(G, (17.269451, 95.583889))
dest_node = ox.get_nearest_node(G, (17.261051, 95.580755))

In [ ]:
# find the route between these nodes then plot it
route = nx.shortest_path(G, orig_node, dest_node, weight='length')
ox.plot_route_folium(G, route)

In [ ]:
# how long is our route in meters?
nx.shortest_path_length(G, orig_node, dest_node, weight='length')

In [ ]:
# how far is it between these two nodes as the crow flies?
ox.great_circle_vec(G.nodes[orig_node]['y'], G.nodes[orig_node]['x'],
                    G.nodes[dest_node]['y'], G.nodes[dest_node]['x'])